# Thai Election Form (S.S. 5/18) Vote Extractor - Google Drive Edition

Extract structured data from Thai election form PDFs using **Gemini with Structured Output**.

**Features:**
- 📁 Direct Google Drive file access (no download needed!)
- 🤖 Uses Gemini models with structured output
- 📊 Pydantic schema validation
- ✅ Automatic validation of extracted data

**Model:** `gemini-exp-1206` or `gemini-2.0-flash-exp` (latest experimental models)

## 1. Setup and Dependencies

In [ ]:
# Install required packages (run once)
# !pip install google-genai pydantic pandas

In [ ]:
import json
import os
from typing import Optional

from google import genai
from google.genai import types
from pydantic import BaseModel, Field
from IPython.display import display, HTML
import pandas as pd

## 2. Configuration

Set your Google Cloud project and Google Drive file ID.

In [ ]:
# Google Cloud Configuration
GOOGLE_CLOUD_PROJECT = os.getenv("GOOGLE_CLOUD_PROJECT", "YOUR_PROJECT_ID")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")

# Model Configuration
MODEL_NAME = "gemini-exp-1206"  # or "gemini-2.0-flash-exp"

# Google Drive file ID
# Example from BigQuery: 1SWv__IY3oqrc4u64OyEV4D9M9K5Awqr6
DRIVE_FILE_ID = "1SWv__IY3oqrc4u64OyEV4D9M9K5Awqr6"

# Construct Google Drive URI
DRIVE_URI = f"https://drive.google.com/uc?export=download&id={DRIVE_FILE_ID}"

print(f"✅ Configuration")
print(f"   Project: {GOOGLE_CLOUD_PROJECT}")
print(f"   API Key: {'*' * 20 + GEMINI_API_KEY[-8:] if GEMINI_API_KEY and len(GEMINI_API_KEY) > 8 else 'NOT SET ⚠️'}")
print(f"   Model: {MODEL_NAME}")
print(f"   Drive File ID: {DRIVE_FILE_ID}")
print(f"   Drive URI: {DRIVE_URI[:60]}...")

if not GEMINI_API_KEY:
    print("\n⚠️  WARNING: GEMINI_API_KEY is not set!")
    print("   Set it with: export GEMINI_API_KEY='your-key-here'")

## 3. Define Pydantic Schema

These models define the structure of data we expect to extract from the election forms.

In [ ]:
class FormInfo(BaseModel):
    """Header information identifying the polling station."""
    date: Optional[str] = Field(None, description="Date of election (e.g., 14 May 2566)")
    province: Optional[str] = Field(None, description="Province name")
    district: str = Field(..., description="District name (Amphoe/Khet)")
    sub_district: Optional[str] = Field(None, description="Sub-district name (Tambon/Khwaeng)")
    constituency_number: Optional[str] = Field(None, description="Constituency number")
    polling_station_number: str = Field(..., description="Unit/Polling Station number")
    form_type: Optional[str] = Field(
        None,
        description="Type of form: Constituency (Candidates) or PartyList (Parties)"
    )


class BallotStatistics(BaseModel):
    """Section 2: Ballot accounting statistics."""
    ballots_allocated: Optional[int] = Field(None, description="Total ballots allocated to station")
    ballots_used: Optional[int] = Field(None, description="Item 2.2: Total used ballots")
    ballots_remaining: Optional[int] = Field(None, description="Ballots remaining/unused")
    good_ballots: Optional[int] = Field(None, description="Item 2.2.1: Valid ballots")
    bad_ballots: Optional[int] = Field(None, description="Item 2.2.2: Void/Spoiled ballots")
    no_vote_ballots: Optional[int] = Field(None, description="Item 2.2.3: No Vote ballots")


class VoteResult(BaseModel):
    """Individual vote result for a candidate or party."""
    number: int = Field(..., description="Candidate/Party Number")
    candidate_name: Optional[str] = Field(None, description="Candidate Name (for Constituency forms)")
    party_name: Optional[str] = Field(None, description="Party Name (for both Constituency and PartyList)")
    vote_count: int = Field(..., description="Vote count (numeric)")
    vote_count_text: Optional[str] = Field(None, description="Vote count (Thai written text)")


class ElectionFormData(BaseModel):
    """Complete election form extraction result."""
    form_info: FormInfo
    ballot_statistics: Optional[BallotStatistics] = None
    vote_results: list[VoteResult] = Field(
        default_factory=list,
        description="Vote counts for all candidates/parties"
    )


print("✅ Pydantic models defined")

## 4. Gemini Schema for Structured Output

Convert Pydantic models to Gemini's schema format.

In [ ]:
# Define the schema for Gemini structured output
ELECTION_DATA_SCHEMA = {
    "type": "ARRAY",
    "description": (
        "A list of election reports found in the PDF "
        "(usually one Constituency report and one PartyList report)."
    ),
    "items": {
        "type": "OBJECT",
        "properties": {
            "form_info": {
                "type": "OBJECT",
                "description": "Header information identifying the polling station and form type.",
                "properties": {
                    "form_type": {
                        "type": "STRING",
                        "enum": ["Constituency", "PartyList"],
                        "description": (
                            "Identify if this specific report is for Constituency (Candidate) "
                            "or PartyList (Party only). Check the header text "
                            "(e.g., 'แบบบัญชีรายชื่อ' = PartyList)."
                        ),
                    },
                    "date": {"type": "STRING", "description": "Date of election"},
                    "province": {"type": "STRING", "description": "Province name"},
                    "district": {"type": "STRING", "description": "District name (Amphoe/Khet)"},
                    "sub_district": {
                        "type": "STRING",
                        "description": "Sub-district name (Tambon/Khwaeng)",
                    },
                    "constituency_number": {"type": "STRING", "description": "Constituency number"},
                    "polling_station_number": {"type": "STRING", "description": "Unit number"},
                },
                "required": ["form_type", "province", "district", "polling_station_number"],
            },
            "ballot_statistics": {
                "type": "OBJECT",
                "description": "Section 2: Ballot accounting for this specific form.",
                "properties": {
                    "ballots_allocated": {
                        "type": "INTEGER",
                        "description": "Item 2.1: Total allocated ballots",
                    },
                    "ballots_used": {
                        "type": "INTEGER",
                        "description": "Item 2.2: Total used ballots",
                    },
                    "good_ballots": {"type": "INTEGER", "description": "Item 2.2.1: Good ballots"},
                    "bad_ballots": {"type": "INTEGER", "description": "Item 2.2.2: Bad ballots"},
                    "no_vote_ballots": {
                        "type": "INTEGER",
                        "description": "Item 2.2.3: No Vote ballots",
                    },
                    "ballots_remaining": {
                        "type": "INTEGER",
                        "description": "Item 2.3: Remaining ballots",
                    },
                },
            },
            "vote_results": {
                "type": "ARRAY",
                "description": "Section 3: Vote counts table from all pages.",
                "items": {
                    "type": "OBJECT",
                    "properties": {
                        "number": {"type": "INTEGER"},
                        "candidate_name": {
                            "type": "STRING",
                            "description": "Name of Candidate (for Constituency). Leave null for PartyList.",
                        },
                        "party_name": {"type": "STRING"},
                        "vote_count": {"type": "INTEGER"},
                        "vote_count_text": {"type": "STRING"},
                    },
                },
            },
        },
        "required": ["form_info", "ballot_statistics", "vote_results"],
    },
}

print("✅ Gemini schema defined")
print(f"   Schema type: {ELECTION_DATA_SCHEMA['type']}")

## 5. Initialize Gemini Client

In [ ]:
# Initialize Gemini client with API key
if not GEMINI_API_KEY:
    raise ValueError(
        "GEMINI_API_KEY is required! Set it with:\n"
        "  export GEMINI_API_KEY='your-key-here'\n"
        "  or create a .env file"
    )

client = genai.Client(
    api_key=GEMINI_API_KEY,
)

print(f"✅ Gemini client initialized (using API key)")
print(f"   Project: {GOOGLE_CLOUD_PROJECT}")
print(f"   Model: {MODEL_NAME}")

## 6. Extraction Function with Google Drive URL

**Key Advantage:** Gemini can directly access PDF files from Google Drive via URL - no download or conversion needed!

In [ ]:
def extract_from_drive_url(
    drive_uri: str,
    model: str = MODEL_NAME,
    temperature: float = 0.0,
    max_tokens: int = 8192,
) -> dict:
    """
    Extract structured vote data from a PDF stored in Google Drive.
    
    Uses Gemini's External URLs file input method.
    Reference: https://ai.google.dev/gemini-api/docs/file-input-methods
    
    Args:
        drive_uri: Google Drive file URI (https://drive.google.com/uc?export=download&id=...)
        model: Gemini model name
        temperature: Sampling temperature (0.0 = deterministic)
        max_tokens: Maximum output tokens
    
    Returns:
        Dictionary containing extracted data
    """
    print(f"🤖 Extracting data with {model}...")
    print(f"   Using Google Drive URI (External URL method)")
    print(f"   URI: {drive_uri[:80]}...")
    
    # Create file part from URI - Gemini will download and process directly
    file_part = types.Part.from_uri(
        file_uri=drive_uri,
        mime_type="application/pdf"
    )
    
    # Extraction prompt
    prompt = """
    You are an expert data entry assistant for Thai Election documents (Form S.S. 5/18).
    
    Instructions:
    1. Analyze all pages of this PDF document.
    2. Extract information strictly according to the JSON schema provided.
    3. Consolidate data from all pages:
       - Header information (District, Date) is usually on the first page
       - Vote Results table often spans multiple pages - merge into single list
    4. Validation: Ensure 'ballots_used' = 'good_ballots' + 'bad_ballots' + 'no_vote_ballots'
    5. Form Type: Detect if 'Constituency' (with candidate names) or 'PartyList' (party only)
    """
    
    # Configure generation with structured output
    generation_config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=ELECTION_DATA_SCHEMA,
        temperature=temperature,
        max_output_tokens=max_tokens,
        top_p=0.95,
        top_k=40,
    )
    
    # Generate content
    print("   Sending request to Gemini...")
    response = client.models.generate_content(
        model=model,
        contents=[file_part, prompt],
        config=generation_config,
    )
    
    # Parse response
    result = json.loads(response.text)
    
    print(f"\n✅ Extraction complete!")
    print(f"   Extracted {len(result)} report(s)")
    
    return result


print("✅ Extraction function defined")

## 7. Validation Functions

In [ ]:
def validate_extraction(data: dict) -> tuple[bool, list[str]]:
    """
    Validate extracted election form data.
    
    Args:
        data: Extracted form data
    
    Returns:
        Tuple of (is_valid, list of error messages)
    """
    errors = []
    
    # Validate ballot statistics
    ballot_stats = data.get("ballot_statistics")
    if ballot_stats:
        ballots_used = ballot_stats.get("ballots_used", 0)
        good = ballot_stats.get("good_ballots", 0)
        bad = ballot_stats.get("bad_ballots", 0)
        no_vote = ballot_stats.get("no_vote_ballots", 0)
        
        expected_total = good + bad + no_vote
        
        if ballots_used != expected_total:
            errors.append(
                f"Ballot mismatch: ballots_used ({ballots_used}) != "
                f"good+bad+no_vote ({expected_total})"
            )
    
    # Validate vote results
    vote_results = data.get("vote_results", [])
    if not vote_results:
        errors.append("No vote results found")
    
    for result in vote_results:
        if result.get("vote_count", 0) < 0:
            name = result.get("candidate_name") or result.get("party_name") or "Unknown"
            errors.append(f"Negative vote count for {name}")
    
    return len(errors) == 0, errors


print("✅ Validation functions defined")

## 8. Display Functions

In [ ]:
def display_results(result: list[dict]):
    """
    Display extraction results in a formatted way.
    
    Args:
        result: List of extracted reports
    """
    if not result:
        print("❌ No data extracted")
        return
    
    # Process each report
    for idx, report in enumerate(result, 1):
        print(f"\n{'='*80}")
        print(f"REPORT #{idx}")
        print(f"{'='*80}")
        
        # Form Information
        form_info = report.get("form_info", {})
        print(f"\n📋 FORM INFORMATION")
        print(f"   Form Type: {form_info.get('form_type', 'N/A')}")
        print(f"   Province: {form_info.get('province', 'N/A')}")
        print(f"   District: {form_info.get('district', 'N/A')}")
        print(f"   Sub-district: {form_info.get('sub_district', 'N/A')}")
        print(f"   Polling Station: {form_info.get('polling_station_number', 'N/A')}")
        print(f"   Constituency: {form_info.get('constituency_number', 'N/A')}")
        print(f"   Date: {form_info.get('date', 'N/A')}")
        
        # Ballot Statistics
        ballot_stats = report.get("ballot_statistics")
        if ballot_stats:
            print(f"\n📦 BALLOT STATISTICS")
            used = ballot_stats.get('ballots_used', 0)
            good = ballot_stats.get('good_ballots', 0)
            bad = ballot_stats.get('bad_ballots', 0)
            no_vote = ballot_stats.get('no_vote_ballots', 0)
            
            print(f"   Allocated: {ballot_stats.get('ballots_allocated', 'N/A'):,}")
            print(f"   Used: {used:,}")
            print(f"   - Good: {good:,}")
            print(f"   - Bad: {bad:,}")
            print(f"   - No Vote: {no_vote:,}")
            print(f"   Remaining: {ballot_stats.get('ballots_remaining', 'N/A'):,}")
            
            # Validation
            expected = good + bad + no_vote
            if used == expected:
                print(f"   ✅ Validation: PASSED ({used:,} = {expected:,})")
            else:
                print(f"   ⚠️  Validation: FAILED ({used:,} ≠ {expected:,})")
        
        # Vote Results Table
        vote_results = report.get("vote_results", [])
        if vote_results:
            print(f"\n📊 VOTE RESULTS ({len(vote_results)} entries)")
            
            # Create DataFrame for nice display
            df_data = []
            for v in vote_results:
                row = {
                    "#": v.get("number"),
                    "Candidate": v.get("candidate_name") or "-",
                    "Party": v.get("party_name") or "-",
                    "Votes": v.get("vote_count", 0),
                }
                df_data.append(row)
            
            df = pd.DataFrame(df_data)
            
            # Display table
            display(df)
            
            # Summary
            total_votes = df["Votes"].sum()
            print(f"\n   Total Votes: {total_votes:,}")


print("✅ Display functions defined")

## 9. Run Extraction

Extract data directly from Google Drive - no download needed!

In [ ]:
# Extract data from Google Drive PDF
result = extract_from_drive_url(
    drive_uri=DRIVE_URI,
    model=MODEL_NAME,
    temperature=0.0,
    max_tokens=8192,
)

## 10. Display Results

In [ ]:
# Display results
display_results(result)

## 11. Save Results

In [ ]:
# Save to JSON file
output_file = f"extracted_data_{DRIVE_FILE_ID}.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"💾 Results saved to: {output_file}")

## 12. Validate with Pydantic

Validate the extracted data against our Pydantic models.

In [ ]:
# Validate each report with Pydantic
for idx, report_data in enumerate(result, 1):
    try:
        # Parse into Pydantic model
        form_data = ElectionFormData(**report_data)
        
        print(f"✅ Report #{idx} - Pydantic validation PASSED")
        print(f"   Form Type: {form_data.form_info.form_type}")
        print(f"   District: {form_data.form_info.district}")
        print(f"   Vote Results: {len(form_data.vote_results)} entries")
        print()
        
    except Exception as e:
        print(f"❌ Report #{idx} - Pydantic validation FAILED")
        print(f"   Error: {e}")
        print()

## 13. Try Different Files

You can easily test with other Google Drive files by changing the file ID.

In [ ]:
# Example: Process a different file
# different_file_id = "ANOTHER_GOOGLE_DRIVE_FILE_ID"
# different_uri = f"https://drive.google.com/uc?export=download&id={different_file_id}"
# 
# result = extract_from_drive_url(different_uri, model=MODEL_NAME)
# display_results(result)

## Summary

This notebook demonstrates the **most efficient workflow** for vote extraction:

✅ **Google Drive Direct Access** - No downloads needed  
✅ **External URLs Method** - Gemini fetches files directly  
✅ **No PDF Conversion** - Gemini handles PDF natively  
✅ **Pydantic Schema** - Type-safe data models  
✅ **Gemini Structured Output** - Guaranteed JSON format  
✅ **Business Logic Validation** - Ballot accounting checks  

### Key Advantages:

1. **No Local Storage** - Files stay in Google Drive
2. **No PDF Conversion** - No pdf2image or poppler needed
3. **Faster** - No download/conversion overhead
4. **Scalable** - Easy to process thousands of files
5. **Cost Effective** - No egress charges

### Integration with BigQuery:

For production workflows, use [gemini-ss5_18_bigquery_drive.ipynb](gemini-ss5_18_bigquery_drive.ipynb) which includes:
- BigQuery integration to find files
- Batch processing capabilities
- Province filtering
- File size limits

### Next Steps:

1. **Test different models** - Compare gemini-exp-1206 vs gemini-2.0-flash-exp
2. **Tune parameters** - Optimize temperature, top_p, top_k
3. **Add error handling** - Retry logic for failed extractions
4. **Scale up** - Process multiple files with BigQuery integration
5. **Save to BigQuery** - Store results back in BigQuery